# Intro, 6/23/25

* DART - Data Assimilation Research Testbed
* NSF NCAR

* Models
  * CAM, WRF, WRF-Chem, MPAS-Atmos, ROMS, MOM6, CICE, NOAH (land), WRF-Hydro, plus some more
 
* Jeff Anderson (NCAR, CISL), funding source

* Tasks
  * Integrate UFO, obs into DART
    * Eric, Steve, Francois
    * IODA format
    * Diagnostics in Skylab
  * DA support fo chemical compo models
    * Jerome, Maryam
   
* 7 Weeks - Jun 23 through Aug 8

* Need to meet with DART team this week
  * Contacts, Mtg schedules, Completion criteria
  * Helen Kershaw - lead SE on DART

* Get a new account key to charge hours

* For now, log into Derecho, try building DART with jedi spack-stack

* Tom will schedule a kickoff meeting

* There exists a converter that's being used to get data into DART
  * Craig Schwartz - Fortran converter
 
* Might be best to use ingest into R2D2, then IODA files into UFO, then output IODA files into obs_seq for DART
  * Think of things to propose to get this done

# Ideas, 6/23/25

* Assume the path is:
  * OBS -> ingest -> R2D2 archive -> R2D2 -> IODA input files -> IODA/UFO -> IODA output file -> IODA to obs_seq converter, plus DART model files -> DART -> DART output
 
* How to bridge IODA output file to what DART needs for input obs
  * Keep netcdf, hdf library versions the same?
  * Is JEDI spack-stack sufficient?
  * OR do we operate with two stacks
    * jedi-spack-stack
    * whatever is being used for DART now
    * This option could allow for netcdf, hdf library versions to be different, but they probably need to be close.

* Sample DART converters
  * Start with netcdf converter for MADIS - convert_madis_profile.f90
  * or SST - oi_sst_to_obs.f90

* github.com/NCAR/DART
  * observations/obs_converters
    * MADIS or SST
  * Fortran code
    * use obs_utilities_mod
    * use obs_sequence_mod
    * use netcdf_utilities_mod
    * use netcdf

# Kickoff with DART team, 6/24/25

* Jeff Anderson
  * Personal goal was to help out with our funding/furlough
  * Goal to collaborate between JEDI, DART is the primary motivation

* Approach this in a very incremental fashion
  * This needs to tie in with incremental funding
 
* COMPO project
  * Want expertise and vision more than lots of implementation

* OBS project
  * Craig Schwartz has been working on something like this
  * Looked a JEDI and started an import path (converter)
  * Project is essentially JEDI observer with DART solver

* DART team is small
  * Interested in tropospheric atmosphere NWP
  * R2O is important
  * Get data from IODA to obs_seg format

* First step (Jeff will setup this meeting)
  * Helen Kershaw (lead SE), FrancoisV, Eric, SteveH
  * Contrast, compare IODA and obs_seq formats
  * Look at IODA to obs_seq converter options

* Jeff - want this to be a collaborative effort. If not going the way you want, speak up

# SE meeting, 6/25/25

* Jeff A
  * SE long time ago, DA scientist now
 
* Helen K
  * SE, 4 years on DART
  * PhD in Mechanical Engineering

* DART
  * Solver
  * File format - "obs sequence"
    * Associated data structure in memory (Fortran linked list)
    * Fortran module (sort of like OO style)
  * Would like to move to netcdf
    * Barrier is the need to attach metadata on the fly
      * Need more clarification on this
  * Converters exist to go into obs_seq format
    * BUFR, netCDF
    * GSI to DART path exists

* Want to leverage as much data resource from JEDI as possible

* Helen
  * Want ease of use
  * python notebook - select obs type, time window -> get the files
  * 40 individual converters - all Fortran

* WIS2
  * WMO replaement for GTS
  * Distributed network
    * Lots of data, all the time

* FrancoisV - obs_seq is an ascii format

* Next steps
  * Helen send us docs, sample files, obs_seg tool
    * Eric and I look at their obs_seq format
    * 10 year global reanalysis (plenty of files)
  * Eric and I report on what we think of how to proceed
    * Step 1 -> ioda to obs_seq converter
    * Step 2 -> JEDI ingest wuite -> IODA to obs_seq -> DART
    * Step 3 -> Look into UFO filter, etc. pre-processing

# SE discussion about IODA, obs_seq formats, 7/3/25

* obs_seq, IODA differences
  * obs_seq is sorted by time
  * obs_seq does not hold metadata for radiances

* DART is driven by obs_seq
  * Depends on obs_seq being sorted by time
  * Pandas will be useful for doing the sort-by-time ordering

* Fortran code includes an "obs_def" module
  * Every obs type has its own obs_def module
 
* Go with a Python converter
  * Extra value in being able to read obs data into a dataframe
  * This is the direction Helen wants to take with converters in general
    * Too much duplicated code with Fortran converters (copy-paste development)

# DART SE, Helen, 7/17/25

* obs_sequence_tool
  * Build mpas_atm
  * This creates a suitable input.nml for the obs_sequence_tool
  * lorenz_63 doesn't work since this uses fabricated ob
* dateTime
  * Assume UTC
  * Use the 1601 reference (start of Gregorian calendar)
  * Use Gregorian calendar
  * days -> number of days since reference
  * seconds -> time of day
  * time -> python datetime string format
* obs_err_var
  * statistical variance of obs error
  * from the data provider
* Data types
  * when build mpas_atm, a list is generated in the input.nml
    * Integer to variable mapping (&obs_kind_nml)
    * This are the names that go into the table in the ObsSequence header
  * Helen will look up if a DART time can be converted to days, seconds
* The obs_seq writer will sort the dataframe, then generate the linked list, then write the file
  * No need to sort the dataframe
* Location
  * Lat: -90 to +90
  * Lon: 0 to 360
* Metadata
  * Only a handful of obs types that need extra metadata
  * RTTOV uses an obs_def to obtain extra metadata
* QC
  * Data_QC is QC from the data provider
    * Equivalent to the IODA PreQC group
  * The user can make up their own name for the QC in the obs_seq.out file
  * DART will use a fixed name for the QC it generated during the obs processing steps
    * "DART quality control"
   
* The issue with the obs sequence scheme that makes it difficult to move to another file format is that the solver piece depends on the obs being presented in chronological order
  * It would take a large effort to remove this dependency, so it's more the solver architecture than the obs_seq file format.
 
* Columns (15) in the dataframe from an obs_seq.out file

| Name | Data Type | Description |
|------|-----------|-------------|
| obs_num | integer | 1..nobs |
| observation | float | obs measurement |
| Data_QC | float | QC marks from data provider |
| linked_list | 3 integers | (prev, next, reserved), internal use |
| longitude | float | 0 to 360 degrees, radians in the file |
| latitude | float | -90 to +90 degrees, radians in the file |
| vertical | float | vertical coordinate value |
| vert_unit | string | vertical coordinate unit |
| type | string | obs type from list in file header |
| metadata | python list | reserved, seems to always be empty for now |
| external_FO | python list | reserved, seems to always be empty for now |
| seconds | integer | time of day |
| days | integer | number of days since 1601-01-01 00:00:00 reference |
| time | python datetime string | absolute time, UTC |
| obs_err_var | float | statistical variarance of obs error |

# Notes, 7/22/25

* Need a better naming convention for the python packages
  * A user looking on PyPI won't know what "pyqceff" and "pyjedi" mean
  * Need a way to indicate DART

* Can the obs_seq layout be done in netcdf?
  * Seems like it could, just have the columns be variables in the netcdf file
  * Can do the linked_list in a variable or a set of variables (prev, next, reserved)
  * This enables compression, new variables, attributes, etc.
  * Can still enforce the temporal ordering

# DART, 7/23/25

* Yannick
  * Some funding has been restored, but not even close to what we have lost
  * Continue with DART is okay
* Jeff A
  * Happy with projects
  * Keep going with projects
* How to validate the radiosonde file produced by the ioda converter
  * Helen has a simple MPAS filter that could check my file
  * Start with a prepBUFR file that can be converter to obs_seq and to ioda
    * Run the ioda2obsq converter and compare with the direct conversion obs_seq file
    * JeffA - if it helps with the comparison the obs_sequence_tool can be used to subset on time ranges and/or lat,lon reggions, etc.

* BenR and FrancoisV will start looking at radiance examples

# Helen, 8/7/25

* RTTOV forward operators
  * IR, UV

* external_FO -> "external forward operator", HofX values from an external source

* Try AMSUA
  * Look at converter docs in DART repo
  * DART/observations/forward_operators
    * obs_def_rttov13_mod.f90
    * Mapping for obs_seq types to DART internal types
    * read_rttov_metadata()
      * reads an obs_seq file (ascii)
      * write the metadata pieces into the metadata column in the same order that this function reads the metadata pieces
      * look at "!temp variables" and order of read statements

* pyDARTdiags
  * Examples:
    * look at RTTOV
    * metadata column
      * each row contains a list

* When building radiance converter, need to build the forward op modules for linking

* Test methodology
  * small data files
  * check in CDL files for the test input and test reference files
  * Tests then call ncdump, ncgen to convert between CDL and netCDF